In [1]:
import pandas as pd
from sqlalchemy import create_engine

def load_data_to_sqlite(file_path, table_name, engine):
    '''Function to load data from DSV files and create tables in SQLite database'''
    # Load data from DSV file with cp1251 encoding
    data = pd.read_csv(file_path, sep='\t', encoding='cp1251')
        
    # Write data to SQLite table
    data.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Create a connection to the SQLite database
engine = create_engine('sqlite:///rag_test_123.db')

# Paths to DSV files and corresponding table names
files_and_tables = {
    'bd/curr_rate.dsv': 'curr_rate',
    'bd/reg.dsv': 'reg',
    'bd/spr.dsv': 'spr'
}

# Load data from all files and create tables in the database
for file_path, table_name in files_and_tables.items():
    load_data_to_sqlite(file_path, table_name, engine)

print("Данные успешно загружены в базу данных SQLite.")

Данные успешно загружены в базу данных SQLite.


In [2]:
tables_desc = '''
Таблицы и их описания:

1. curr_rate:
- STAT_MONTH: Месяц статистики
- STAT_YEAR: Год статистики
- USD: Курс доллара
- EURO: Курс евро

2. data:
- STAT_MONTH: Месяц статистики
- STAT_YEAR: Год статистики
- STAT_DATE: Дата статистики
- CD_REG: Код региона
- CD_U: Код пользователя
- PRCAVG_W_IN: Средняя цена W на входе
- PRCAVG_W_OUT: Средняя цена W на выходе
- PRCAVG_A_IN: Средняя цена A на входе
- PRCAVG_A_OUT: Средняя цена A на выходе
- PRCMIN_IN: Минимальная цена на входе
- PRCMIN_OUT: Минимальная цена на выходе
- PRCMAX_IN: Максимальная цена на входе
- PRCMAX_OUT: Максимальная цена на выходе
- PRCMED_IN: Медианная цена на входе
- PRCMED_OUT: Медианная цена на выходе
- VOLSHT_OUT: Объем на выходе (штуки)
- VOLRUB_OUT: Объем на выходе (рубли)
- PRED_PN_FIRM: Прогноз Pn фирмы
- PRED_TN: Прогноз TN
- PRED_TN_FIRM: Прогноз TN фирмы

3. reg:
- CD_REG: Код региона
- NM_REG: Название региона
- NM_REG_LAT: Название региона (латиница)
- LEV: Уровень
- PARENT_CD_REG: Код родительского региона
- CHANNEL: Канал

4. spr:
- CD_U: Код пользователя
- CD_IAS: Код IAS
- NM_FULL: Полное название
- NM_T: Краткое название
- NM_BR: Название бренда
- GROUP_NM_RUS: Группа на русском
- NM_C: Название C
- NM_F: Название F
- RECIPEREQ: Требование рецепта
- ATC1: Код ATC1
- ATC2: Код ATC2
- ATC3: Код ATC3
- ATC4: Код ATC4
- ATC5: Код ATC5
- EPHMRA1: Код EPHMRA1
- EPHMRA2: Код EPHMRA2
- EPHMRA3: Код EPHMRA3
- EPHMRA4: Код EPHMRA4
- BAD1: Код BAD1
- NM_BAD1: Название BAD1
- BAD2: Код BAD2
- NM_BAD2: Название BAD2
- BAD_A_1: Код BAD_A_1
- NM_BAD_A_1: Название BAD_A_1
- BAD_A_2: Код BAD_A_2
- NM_BAD_A_2: Название BAD_A_2
- BAD_A_3: Код BAD_A_3
- NM_BAD_A_3: Название BAD_A_3
- INN: INN
- COUNT_BL: Количество BL
- COUNT_IN_BL: Количество в BL
- MV: MV
- MV_NM_MU: MV NM MU
- NM_DOMESTIC: Название на родном языке
- NM_DT: Название DT
- NM_PACK: Название упаковки

'''

In [3]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage
from huggingface_hub import login

# Huggingface access token
login(token = 'your_huggingface_token')
# MistralAI API
MISTRAL_API_KEY = "your_API_key"

# MistralAI model for embeddings
llm = ChatMistralAI(api_key=MISTRAL_API_KEY)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Chiki\.cache\huggingface\token
Login successful


In [4]:
from langchain_core.prompts.chat import ChatPromptTemplate

template_query = """Generate a SQL query to answer this question: {question}\nDDL statements (use only column names from this schema): {schema}\nThe following SQL query best answers the question:"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        ("human", template_query),
    ]
)

In [5]:
def get_sql_answer(query, schema):
    chain = prompt.pipe(llm)
    return chain.invoke({
            'schema': schema,
            'question': query
        }).content

In [6]:
from sqlalchemy import text

def execute_query(query, engine):
    with engine.connect() as connection:
        result = connection.execute(text(query))
        return result.fetchall()

def get_data_from_sql(sql_code, engine):
    return execute_query(sql_code, engine)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

model_name = 'Qwen/Qwen2-1.5B-Instruct'
device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model_vector = AutoModel.from_pretrained(model_name).to(device)
device = model.device
print(f'device = {device}')

In [7]:
from datetime import datetime
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

def vectorize_text(text):
    if len(text.strip()) == 0:
        return None 
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model_vector(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def generate_answer(context, query):
    messages = [
    {"role": "system", "content": f"Ты помощник директора компании Эвалар, сегодня дата {datetime.date}, тебе требуется ответить на вопрос пользователя имея данные из базы данных: {context}"},
    {"role": "user", "content": query}
]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt") # .to(device)
    attention_mask = model_inputs["attention_mask"] # .to(device)
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        attention_mask=attention_mask,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [8]:
def get_answer(query, logging=False):
    sql_code=get_sql_answer(query, tables_desc).replace('\\', '') # почему-то mistralai добавляет \\ в некоторых местах
    
    print(f'Generated SQL code: \n{sql_code}', end='\n\n')
    
    sql_context = get_data_from_sql(sql_code=sql_code, engine=engine)
    context = f'SQL Code: {sql_code}, SQL Output: '
    for i in sql_context:
        context += f'{i} '
        
    if logging:
        print(f'SQL context: \n{sql_context}', end='\n\n')
        
    answer = generate_answer(context, query)
    return answer

In [9]:
query = "В какое время был самый большой курс доллара"
get_answer(query, True)

Generated SQL code: 
SELECT STAT_YEAR, STAT_MONTH, MAX(USD) AS MaxDollarRate
FROM curr_rate
GROUP BY STAT_YEAR, STAT_MONTH
ORDER BY MaxDollarRate DESC
LIMIT 1;

SQL context: 
[(2022, 3, 104.080968)]

